In [1]:
import numpy as np
import pandas as pd
import os
import copy
import operator

In [2]:
os.listdir('./data/partition')

['.DS_Store',
 '.Rhistory',
 'ForecastDataforTesting_201712_10.csv',
 'ForecastDataforTesting_201712_6.csv',
 'ForecastDataforTesting_201712_7.csv',
 'ForecastDataforTesting_201712_8.csv',
 'ForecastDataforTesting_201712_9.csv',
 'ForecastDataforTraining_201712_1.csv',
 'ForecastDataforTraining_201712_2.csv',
 'ForecastDataforTraining_201712_3.csv',
 'ForecastDataforTraining_201712_4.csv',
 'ForecastDataforTraining_201712_5.csv',
 'In_situMeasurementforTraining_201712_1.csv',
 'In_situMeasurementforTraining_201712_2.csv',
 'In_situMeasurementforTraining_201712_3.csv',
 'In_situMeasurementforTraining_201712_4.csv',
 'In_situMeasurementforTraining_201712_5.csv']

In [3]:
df = pd.read_csv('./data/partition/ForecastDataforTesting_201712_6.csv')


In [4]:
df = df.groupby(['xid', 'yid', 'date_id', 'hour']).mean()

In [5]:
df = df.reset_index()

In [6]:
df.dtypes

xid          int64
yid          int64
date_id      int64
hour         int64
model      float64
wind       float64
dtype: object

In [7]:
df = df.drop('model', axis=1)

In [8]:
os.listdir('./data')

['.DS_Store', 'CityData.csv', 'partition']

In [9]:
df_city = pd.read_csv('data/CityData.csv')

In [10]:
df_city

,cid,xid,yid
0,0,142,328
1,1,84,203
2,2,199,371
3,3,140,234
4,4,236,241
5,5,315,281
6,6,358,207
7,7,363,237
8,8,423,266
9,9,125,375


In [11]:
df.loc[:, ['xid', 'yid']].drop_duplicates(keep='first').shape

(230708, 2)

In [12]:
df.loc[:, ['xid', 'yid']].drop_duplicates(keep='first').apply([min, max], axis=0)

,xid,yid
min,1,1
max,548,421


In [13]:
df['hour'].unique()

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20])

In [14]:
df['hour'] = df['hour']*60-180

In [15]:
df['hour'].unique()

array([   0,   60,  120,  180,  240,  300,  360,  420,  480,  540,  600,
        660,  720,  780,  840,  900,  960, 1020])

In [16]:
def find_neighbors(x, limit_x=[1, 548], limit_y=[1, 421]):
    xx = [(x[0]+i_i, x[1]+i_j) for i_i in [-1, 0, 1] for i_j in [-1, 0, 1] if abs(i_i)+abs(i_j)==1]
    
    xx = list(filter(lambda i: i[0]>=limit_x[0] and i[0]<=limit_x[1] and i[1]>=limit_y[0] and i[1]<=limit_y[1], xx))
    return xx

In [17]:
import numpy as np
import functools
import operator


def _reconcile(s1, s2):
    pts = np.unique(np.sort(np.concatenate([s1._x, s2._x])))
    # Handle case when endpoints are inf
    cpts = pts.copy()
    cpts[0] = min(np.min(cpts[1:]), 0.) - 1
    cpts[-1] = max(np.max(cpts[:-1]), 0.) + 1
    mps = (cpts[1:] + cpts[:-1]) / 2.
    return [(pts, s(mps)) for s in (s1, s2)]


def _same_support(s1, s2):
    return np.all(s1._x[[0, -1]] == s2._x[[0, -1]])


def require_compatible(f):
    @functools.wraps(f)
    def wrapper(self, other, *args, **kwargs):
        if isinstance(other, StepFunction) and not _same_support(self, other):
            raise TypeError("Step functions have different support: %s vs. %s" % (
                self._x[[0, -1]], other._x[[0, -1]]))
        return f(self, other, *args, **kwargs)
    return wrapper


class StepFunction:
    '''A step function.'''

    def __init__(self, x, y):
        '''Initialize step function with breakpoints x and function values y.
        x and y are arrays such that
            f(z) = y[k], x[k] <= z < x[k + 1], 0 <= k < K.
        Thus, len(x) == len(y) + 1 and domain of f is (x[0], x[K]).
        '''
        if len(x) != 1 + len(y):
            raise RuntimeError("len(x) != 1 + len(y)")
        self._x = np.array(x)
        self._y = np.array(y)
        self._compress()

    @property
    def K(self):
        '''The number of steps.'''
        return len(self._y)

    def _compress(self):
        # Combine steps which have equal values
        ny = np.concatenate([[np.nan], self._y, [np.nan]])
        ys = np.diff(ny) != 0
        self._x = self._x[ys]
        self._y = self._y[ys[:-1]]

    def _binary_op(self, other, op, desc):
        if isinstance(other, StepFunction):
            (s1_x, s1_y), (s2_x, s2_y) = _reconcile(self, other)
            return StepFunction(s1_x, op(s1_y, s2_y))
        # Fall back to normal semantics otherwise
        return StepFunction(self._x, op(self._y, other))

    def __add__(self, other):
        return self._binary_op(other, operator.add, "add")

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        return self._binary_op(other, operator.sub, "subtract")

    def __rsub__(self, other):
        return -self + other

    def __mul__(self, other):
        return self._binary_op(other, operator.mul, "multiply")

    def __rmul__(self, other):
        return self * other

    def __div__(self, other):
        return self._binary_op(other, operator.div, "divide")

    def __rdiv__(self, other):
        return (self ** -1) * other

    # Unary operations

    def __neg__(self):
        return StepFunction(self._x, -self._y)

    def __pow__(self, p):
        return StepFunction(self._x, pow(self._y, p))

    def __abs__(self):
        return StepFunction(self._x, abs(self._y))

    # Equality and comparison operators

    @require_compatible
    def __eq__(self, other):
        return (np.array_equal(self._x, other._x) and 
                np.array_equal(self._y, other._y))

    @require_compatible
    def __lt__(self, other):
        diff = other - self
        return np.all(diff._y > 0)

    @require_compatible
    def __le__(self, other):
        diff = other - self
        return np.all(diff._y >= 0)

    @require_compatible
    def __gt__(self, other):
        return -self < -other

    @require_compatible
    def __ge__(self, other):
        return -self <= -other

    def __call__(self, s):
        return self._y[np.searchsorted(self._x, s, side="right") - 1]

    def __repr__(self):
        return "StepFunction(x=%s, y=%s)" % (repr(self._x), repr(self._y))

    def integral(self):
        nz = self._y != 0
        d = np.diff(self._x)
        return (d[nz] * self._y[nz]).sum()
    
    def find_interval_value(self, x_interval):
        # [a, b)
        y = np.arange(np.argwhere(self._x<=x_interval[0])[-1], np.argwhere(self._x<x_interval[1])[-1]+1)
        yy = self._y[y]
        yyy = np.sort(np.unique(np.append(self._x[y], x_interval)))
        yyy = yyy[np.all(np.array([yyy>=x_interval[0], yyy<=x_interval[1]]), axis=0)]
        return np.array([yyy, yy])

In [18]:
def hvdistance(x, y):
    return np.sum(np.abs(np.array(x) - np.array(y)))

In [22]:
# In all of the following, the list of intervals must be sorted and 
# non-overlapping. We also assume that the intervals are half-open, so
# that x is in tp(start, end) iff start <= x and x < end.
class tp():
   def __repr__(self):
       return '(%.2f,%.2f)' % (self.start, self.end)
   def __init__(self,start,end): 
       self.start=start
       self.end=end
    
def flatten(list_of_tps):
  """Convert a list of intervals to a list of endpoints"""
  return reduce(lambda ls, ival: ls + [ival.start, ival.end],
                list_of_tps,
                [])

def unflatten(list_of_endpoints):
  """Convert a list of endpoints, with an optional terminating sentinel,
     into a list of intervals"""
  return [tp(list_of_endpoints[i], list_of_endpoints[i + 1])
          for i in range(0, len(list_of_endpoints) - 1, 2)]

def merge(a_tps, b_tps, op):
  """Merge two lists of intervals according to the boolean function op"""
  a_endpoints = flatten(a_tps)
  b_endpoints = flatten(b_tps)

  sentinel = max(a_endpoints[-1], b_endpoints[-1]) + 1
  a_endpoints += [sentinel]
  b_endpoints += [sentinel]

  a_index = 0
  b_index = 0

  res = []

  scan = min(a_endpoints[0], b_endpoints[0])
  while scan < sentinel:
    in_a = not ((scan < a_endpoints[a_index]) ^ (a_index % 2))
    in_b = not ((scan < b_endpoints[b_index]) ^ (b_index % 2))
    in_res = op(in_a, in_b)

    if in_res ^ (len(res) % 2): res += [scan]
    if scan == a_endpoints[a_index]: a_index += 1
    if scan == b_endpoints[b_index]: b_index += 1
    scan = min(a_endpoints[a_index], b_endpoints[b_index])

  return unflatten(res)

def interval_diff(a, b):
  return merge(a, b, lambda in_a, in_b: in_a and not in_b)

def interval_union(a, b):
  return merge(a, b, lambda in_a, in_b: in_a or in_b)

def interval_intersect(a, b):
  return merge(a, b, lambda in_a, in_b: in_a and in_b)

In [23]:
class vertex_i:
    def __init__(self, vi, vs, td, ta):
        self.vi = vi
        self.Ti = np.array([hvdistance(vi, vs), ta])
        self.Si = np.array([ta, ta])
        self.TAUi = None
        self.negihbor_Sij = {}
        self.ta = ta

        
    #def update_Si(self):
        
    def update_TAUi_Si(self, x):
        f = StepFunction(self.gi_d, self.gi_v)
        y = f.find_interval_value(x)
        self.TAUi = y[1].min()
        self.Si = np.array([y[0][np.argmin(y[1])], self.ta])
    
    def create_negihbor(self, ta):
        
        x = find_neighbors(self.vi, limit_x=[1, 548], limit_y=[1, 421])
        y = [[ta-2, ta-2]]*len(x)
        
        self.negihbor_Sij = dict(zip(x, y))
    
    def create_gi(self):
        self.gi_d = [0, self.ta]
        self.gi_v = [float('Inf')]
    
    def update_gi(self, i_x, i_y):
        f = StepFunction(self.gi_d, self.gi_v)
        x = np.sort(np.unique(np.append(self.gi_d, i_x)))
        x = np.array([[x[i], x[i+1]] for i in range(0, len(x)-1)])
        self.gi_d = np.unique(x.flatten())
        self.gi_v = [min(f.find_interval_value(i)[1][0], i_y) if i[0]>=i_x[0] and i[1]<=i_x[1] else f.find_interval_value(i)[1][0] for i in x]
        

In [28]:

def compute_minimum_cost(v_s, v_e, t_d, t_a):

    Q = {}
    QQ = []
    v_i = vertex_i(v_s, v_s, t_d, t_a)
    v_i.gi_d = [t_d, t_a]
    v_i.gi_v = [0]
    v_i.update_TAUi_Si(v_i.Ti)
    v_i.create_negihbor(t_a)
    Q.update({v_s: v_i})
    while v_i.vi != v_e:
        
        for i in find_neighbors(v_i.vi, limit_x=[1, 548], limit_y=[1, 421]):
            if v_i.Si[0] <= (t_a - 2):
                
                R_ij = [v_i.Si[0], t_a-2]
                 
                x = interval_diff([tp(R_ij[0], R_ij[1])], [tp(v_i.negihbor_Sij[i][0],v_i.negihbor_Sij[i][1])])
              
                domain_ij = [x[0].start+2, x[0].end+2]
        
        
                v_i.negihbor_Sij[i] = R_ij
                
                f_ij = df[(df['xid'] == i[0]) & (df['yid'] == i[1])]
                f_ij.sort_values(by='hour', ascending=True, inplace=True)
                
                f_ij_d = np.append(f_ij.hour, t_a)
                
                
                f_ij_v = np.where(f_ij.wind>=15, 1440, 0)
                
                if i not in [iii.vi for iii in Q]:
                    v_j = vertex_i(i, v_s, t_d, t_a)
                    v_j.create_gi()
                    v_j.create_negihbor(t_a)
                else:
                    v_j = [iii for iii in Q if iii.vi == i][0]
                    
                for jj in range(0, len(f_ij_d)-1):
                    i_x = [f_ij_d[jj], f_ij_d[jj+1]]
                    i_y = f_ij_v[jj] + v_i.TAUi
                    
                    v_j.update_gi(i_x, i_y)
                
                x = interval_diff([tp(v_j.Ti[0], v_j.Ti[1])], [tp(v_j.Si[0], v_j.Si[1])])
                print x
                for ii in x:
                    v_j.update_TAUi_Si([ii.start, ii.end])
                
                if i not in [iii.vi for iii in Q]:
                    Q.append(copy.copy(v_j))
                del v_j
                
                
        if (v_i.Ti[0] != v_i.Si[0]) or (v_i.Ti[1] != v_i.Si[1]):
            x = interval_diff([tp(v_i.Ti[0], v_i.Ti[1])], [tp(v_i.Si[0], v_i.Si[1])])
            for ii in x:
                v_i.update_TAUi_Si([ii.start, ii.end])
            
            QQ.append(copy.copy())
            del v_i
        
        Q = sorted(Q, key=lambda ob: ob.TAUi, reverse=False)
        
        v_i = Q.pop(0)
    return v_i      
            
    
    

In [50]:

def compute_minimum_cost(v_s, v_e, t_d, t_a):

    Q = {}
    QQ = []
    Q[v_s] = vertex_i(v_s, v_s, t_d, t_a)
    Q[v_s].gi_v = [0]
    Q[v_s].gi_d = [t_d, t_a]
    t = Q[v_s].Ti
    Q[v_s].update_TAUi_Si(t)
    Q[v_s].create_negihbor(t_a)
    
    
    Q[v_s].gi_d = [t_d, t_a]
    v_i = v_s
    
    while v_i != v_e:
        
        for i in find_neighbors(v_i, limit_x=[1, 548], limit_y=[1, 421]):
            if Q[v_i].Si[0] <= (t_a - 2):
                
                R_ij = [Q[v_i].Si[0], t_a-2]
                 
                x = interval_diff([tp(R_ij[0], R_ij[1])], [tp(Q[v_i].negihbor_Sij[i][0],Q[v_i].negihbor_Sij[i][1])])
              
                domain_ij = [x[0].start+2, x[0].end+2]
        
        
                Q[v_i].negihbor_Sij[i] = R_ij
                
                f_ij = df[(df['xid'] == i[0]) & (df['yid'] == i[1])]
                f_ij.sort_values(by='hour', ascending=True, inplace=True)
                
                f_ij_d = np.append(f_ij.hour, t_a)
                
                
                f_ij_v = np.where(f_ij.wind>=15, 1440, 0)
                
                if i not in [iii for iii in Q]:
                    Q[i] = vertex_i(i, v_s, t_d, t_a)
                    Q[i].create_gi()
                    Q[i].create_negihbor(t_a)
 
                x = interval_diff([tp(Q[i].Ti[0], Q[i].Ti[1])], [tp(Q[i].Si[0], Q[i].Si[1])])
                print QQ
                if len(x)>0:
                    for jj in range(0, len(f_ij_d)-1):
                        i_x = [f_ij_d[jj], f_ij_d[jj+1]]
                        i_y = f_ij_v[jj] + Q[v_i].TAUi
                    
                        Q[i].update_gi(i_x, i_y)
                
                
                    for ii in x:
                        Q[i].update_TAUi_Si([ii.start, ii.end])
                
                    if i not in [iii for iii, jjj in QQ]:
                        QQ.append((i, Q[i].TAUi))
               
                
                
                
                
        if (Q[v_i].Ti[0] != Q[v_i].Si[0]) or (Q[v_i].Ti[1] != Q[v_i].Si[1]):
            x = interval_diff([tp(Q[v_i].Ti[0], Q[v_i].Ti[1])], [tp(Q[v_i].Si[0], Q[v_i].Si[1])])
            for ii in x:
                Q[v_i].update_TAUi_Si([ii.start, ii.end])
            
            QQ.append((v_i, Q[v_i].TAUi))
            
        
        QQ = sorted(QQ, key=operator.itemgetter(1), reverse=False)
        
        v_i = QQ.pop(0)[0]
    return Q      
            
    
    

In [51]:
p_result = compute_minimum_cost((50, 50), (55, 55), 0, 1080)

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[]
[((49, 50), 0.0)]
[((49, 50), 0.0), ((50, 49), 0.0)]
[((49, 50), 0.0), ((50, 49), 0.0), ((50, 51), 0.0)]
[((50, 49), 0.0), ((50, 51), 0.0), ((51, 50), 0.0)]
[((50, 49), 0.0), ((50, 51), 0.0), ((51, 50), 0.0), ((48, 50), 0.0)]
[((50, 49), 0.0), ((50, 51), 0.0), ((51, 50), 0.0), ((48, 50), 0.0), ((49, 49), 0.0)]
[((50, 49), 0.0), ((50, 51), 0.0), ((51, 50), 0.0), ((48, 50), 0.0), ((49, 49), 0.0), ((49, 51), 0.0)]
[((50, 51), 0.0), ((51, 50), 0.0), ((48, 50), 0.0), ((49, 49), 0.0), ((49, 51), 0.0)]
[((50, 51), 0.0), ((51, 50), 0.0), ((48, 50), 0.0), ((49, 49), 0.0), ((49, 51), 0.0)]
[((50, 51), 0.0), ((51, 50), 0.0), ((48, 50), 0.0), ((49, 49), 0.0), ((49, 51), 0.0), ((50, 48), 0.0)]
[((50, 51), 0.0), ((51, 50), 0.0), ((48, 50), 0.0), ((49, 49), 0.0), ((49, 51), 0.0), ((50, 48), 0.0)]
[((51, 50), 0.0), ((48, 50), 0.0), ((49, 49), 0.0), ((49, 51), 0.0), ((50, 48), 0.0), ((51, 49), 0.0)]
[((51, 50), 0.0), ((48, 50), 0.0), ((49, 49), 0.0), ((49, 51), 0.0), ((50, 48), 0.0), ((51, 49), 0.0)

[((49, 48), 0.0), ((49, 52), 0.0), ((50, 47), 0.0), ((51, 48), 0.0), ((52, 49), 0.0), ((50, 53), 0.0), ((51, 52), 0.0), ((52, 51), 0.0), ((53, 50), 0.0), ((46, 50), 0.0), ((47, 49), 0.0), ((47, 51), 0.0), ((48, 48), 0.0)]
[((49, 48), 0.0), ((49, 52), 0.0), ((50, 47), 0.0), ((51, 48), 0.0), ((52, 49), 0.0), ((50, 53), 0.0), ((51, 52), 0.0), ((52, 51), 0.0), ((53, 50), 0.0), ((46, 50), 0.0), ((47, 49), 0.0), ((47, 51), 0.0), ((48, 48), 0.0)]
[((49, 48), 0.0), ((49, 52), 0.0), ((50, 47), 0.0), ((51, 48), 0.0), ((52, 49), 0.0), ((50, 53), 0.0), ((51, 52), 0.0), ((52, 51), 0.0), ((53, 50), 0.0), ((46, 50), 0.0), ((47, 49), 0.0), ((47, 51), 0.0), ((48, 48), 0.0), ((48, 52), 0.0)]
[((49, 52), 0.0), ((50, 47), 0.0), ((51, 48), 0.0), ((52, 49), 0.0), ((50, 53), 0.0), ((51, 52), 0.0), ((52, 51), 0.0), ((53, 50), 0.0), ((46, 50), 0.0), ((47, 49), 0.0), ((47, 51), 0.0), ((48, 48), 0.0), ((48, 52), 0.0)]
[((49, 52), 0.0), ((50, 47), 0.0), ((51, 48), 0.0), ((52, 49), 0.0), ((50, 53), 0.0), ((51, 52)

[((46, 50), 0.0), ((47, 49), 0.0), ((47, 51), 0.0), ((48, 48), 0.0), ((48, 52), 0.0), ((49, 47), 0.0), ((49, 53), 0.0), ((50, 46), 0.0), ((51, 47), 0.0), ((52, 48), 0.0), ((53, 49), 0.0), ((50, 54), 0.0), ((51, 53), 0.0), ((52, 52), 0.0), ((53, 51), 0.0)]
[((46, 50), 0.0), ((47, 49), 0.0), ((47, 51), 0.0), ((48, 48), 0.0), ((48, 52), 0.0), ((49, 47), 0.0), ((49, 53), 0.0), ((50, 46), 0.0), ((51, 47), 0.0), ((52, 48), 0.0), ((53, 49), 0.0), ((50, 54), 0.0), ((51, 53), 0.0), ((52, 52), 0.0), ((53, 51), 0.0)]
[((46, 50), 0.0), ((47, 49), 0.0), ((47, 51), 0.0), ((48, 48), 0.0), ((48, 52), 0.0), ((49, 47), 0.0), ((49, 53), 0.0), ((50, 46), 0.0), ((51, 47), 0.0), ((52, 48), 0.0), ((53, 49), 0.0), ((50, 54), 0.0), ((51, 53), 0.0), ((52, 52), 0.0), ((53, 51), 0.0)]
[((46, 50), 0.0), ((47, 49), 0.0), ((47, 51), 0.0), ((48, 48), 0.0), ((48, 52), 0.0), ((49, 47), 0.0), ((49, 53), 0.0), ((50, 46), 0.0), ((51, 47), 0.0), ((52, 48), 0.0), ((53, 49), 0.0), ((50, 54), 0.0), ((51, 53), 0.0), ((52, 52),

[((51, 47), 0.0), ((52, 48), 0.0), ((53, 49), 0.0), ((50, 54), 0.0), ((51, 53), 0.0), ((52, 52), 0.0), ((53, 51), 0.0), ((54, 50), 0.0), ((45, 50), 0.0), ((46, 49), 0.0), ((46, 51), 0.0), ((47, 48), 0.0), ((47, 52), 0.0), ((48, 47), 0.0), ((48, 53), 0.0), ((49, 46), 0.0), ((49, 54), 0.0)]
[((51, 47), 0.0), ((52, 48), 0.0), ((53, 49), 0.0), ((50, 54), 0.0), ((51, 53), 0.0), ((52, 52), 0.0), ((53, 51), 0.0), ((54, 50), 0.0), ((45, 50), 0.0), ((46, 49), 0.0), ((46, 51), 0.0), ((47, 48), 0.0), ((47, 52), 0.0), ((48, 47), 0.0), ((48, 53), 0.0), ((49, 46), 0.0), ((49, 54), 0.0)]
[((51, 47), 0.0), ((52, 48), 0.0), ((53, 49), 0.0), ((50, 54), 0.0), ((51, 53), 0.0), ((52, 52), 0.0), ((53, 51), 0.0), ((54, 50), 0.0), ((45, 50), 0.0), ((46, 49), 0.0), ((46, 51), 0.0), ((47, 48), 0.0), ((47, 52), 0.0), ((48, 47), 0.0), ((48, 53), 0.0), ((49, 46), 0.0), ((49, 54), 0.0), ((50, 45), 0.0)]
[((51, 47), 0.0), ((52, 48), 0.0), ((53, 49), 0.0), ((50, 54), 0.0), ((51, 53), 0.0), ((52, 52), 0.0), ((53, 51),

...

[((50, 40), 0.0), ((51, 41), 0.0), ((52, 42), 0.0), ((53, 43), 0.0), ((54, 44), 0.0), ((55, 45), 0.0), ((56, 46), 0.0), ((57, 47), 0.0), ((58, 48), 0.0), ((59, 49), 0.0), ((50, 60), 0.0), ((51, 59), 0.0), ((52, 58), 0.0), ((53, 57), 0.0), ((54, 56), 0.0), ((55, 55), 0.0), ((56, 54), 0.0), ((57, 53), 0.0), ((58, 52), 0.0), ((59, 51), 0.0), ((60, 50), 0.0), ((39, 50), 0.0), ((40, 49), 0.0), ((40, 51), 0.0), ((41, 48), 0.0), ((41, 52), 0.0), ((42, 47), 0.0), ((42, 53), 0.0), ((43, 46), 0.0), ((43, 54), 0.0), ((44, 45), 0.0), ((44, 55), 0.0), ((45, 44), 0.0), ((45, 56), 0.0), ((46, 43), 0.0), ((46, 57), 0.0), ((47, 42), 0.0), ((47, 58), 0.0), ((48, 41), 0.0), ((48, 59), 0.0), ((49, 40), 0.0), ((49, 60), 0.0)]
[((51, 41), 0.0), ((52, 42), 0.0), ((53, 43), 0.0), ((54, 44), 0.0), ((55, 45), 0.0), ((56, 46), 0.0), ((57, 47), 0.0), ((58, 48), 0.0), ((59, 49), 0.0), ((50, 60), 0.0), ((51, 59), 0.0), ((52, 58), 0.0), ((53, 57), 0.0), ((54, 56), 0.0), ((55, 55), 0.0), ((56, 54), 0.0), ((57, 53), 0

[((54, 44), 0.0), ((55, 45), 0.0), ((56, 46), 0.0), ((57, 47), 0.0), ((58, 48), 0.0), ((59, 49), 0.0), ((50, 60), 0.0), ((51, 59), 0.0), ((52, 58), 0.0), ((53, 57), 0.0), ((54, 56), 0.0), ((55, 55), 0.0), ((56, 54), 0.0), ((57, 53), 0.0), ((58, 52), 0.0), ((59, 51), 0.0), ((60, 50), 0.0), ((39, 50), 0.0), ((40, 49), 0.0), ((40, 51), 0.0), ((41, 48), 0.0), ((41, 52), 0.0), ((42, 47), 0.0), ((42, 53), 0.0), ((43, 46), 0.0), ((43, 54), 0.0), ((44, 45), 0.0), ((44, 55), 0.0), ((45, 44), 0.0), ((45, 56), 0.0), ((46, 43), 0.0), ((46, 57), 0.0), ((47, 42), 0.0), ((47, 58), 0.0), ((48, 41), 0.0), ((48, 59), 0.0), ((49, 40), 0.0), ((49, 60), 0.0), ((50, 39), 0.0), ((51, 40), 0.0), ((52, 41), 0.0), ((53, 42), 0.0)]
[((54, 44), 0.0), ((55, 45), 0.0), ((56, 46), 0.0), ((57, 47), 0.0), ((58, 48), 0.0), ((59, 49), 0.0), ((50, 60), 0.0), ((51, 59), 0.0), ((52, 58), 0.0), ((53, 57), 0.0), ((54, 56), 0.0), ((55, 55), 0.0), ((56, 54), 0.0), ((57, 53), 0.0), ((58, 52), 0.0), ((59, 51), 0.0), ((60, 50), 0

[((57, 47), 0.0), ((58, 48), 0.0), ((59, 49), 0.0), ((50, 60), 0.0), ((51, 59), 0.0), ((52, 58), 0.0), ((53, 57), 0.0), ((54, 56), 0.0), ((55, 55), 0.0), ((56, 54), 0.0), ((57, 53), 0.0), ((58, 52), 0.0), ((59, 51), 0.0), ((60, 50), 0.0), ((39, 50), 0.0), ((40, 49), 0.0), ((40, 51), 0.0), ((41, 48), 0.0), ((41, 52), 0.0), ((42, 47), 0.0), ((42, 53), 0.0), ((43, 46), 0.0), ((43, 54), 0.0), ((44, 45), 0.0), ((44, 55), 0.0), ((45, 44), 0.0), ((45, 56), 0.0), ((46, 43), 0.0), ((46, 57), 0.0), ((47, 42), 0.0), ((47, 58), 0.0), ((48, 41), 0.0), ((48, 59), 0.0), ((49, 40), 0.0), ((49, 60), 0.0), ((50, 39), 0.0), ((51, 40), 0.0), ((52, 41), 0.0), ((53, 42), 0.0), ((54, 43), 0.0), ((55, 44), 0.0), ((56, 45), 0.0)]
[((57, 47), 0.0), ((58, 48), 0.0), ((59, 49), 0.0), ((50, 60), 0.0), ((51, 59), 0.0), ((52, 58), 0.0), ((53, 57), 0.0), ((54, 56), 0.0), ((55, 55), 0.0), ((56, 54), 0.0), ((57, 53), 0.0), ((58, 52), 0.0), ((59, 51), 0.0), ((60, 50), 0.0), ((39, 50), 0.0), ((40, 49), 0.0), ((40, 51), 0

[((51, 59), 0.0), ((52, 58), 0.0), ((53, 57), 0.0), ((54, 56), 0.0), ((55, 55), 0.0), ((56, 54), 0.0), ((57, 53), 0.0), ((58, 52), 0.0), ((59, 51), 0.0), ((60, 50), 0.0), ((39, 50), 0.0), ((40, 49), 0.0), ((40, 51), 0.0), ((41, 48), 0.0), ((41, 52), 0.0), ((42, 47), 0.0), ((42, 53), 0.0), ((43, 46), 0.0), ((43, 54), 0.0), ((44, 45), 0.0), ((44, 55), 0.0), ((45, 44), 0.0), ((45, 56), 0.0), ((46, 43), 0.0), ((46, 57), 0.0), ((47, 42), 0.0), ((47, 58), 0.0), ((48, 41), 0.0), ((48, 59), 0.0), ((49, 40), 0.0), ((49, 60), 0.0), ((50, 39), 0.0), ((51, 40), 0.0), ((52, 41), 0.0), ((53, 42), 0.0), ((54, 43), 0.0), ((55, 44), 0.0), ((56, 45), 0.0), ((57, 46), 0.0), ((58, 47), 0.0), ((59, 48), 0.0), ((60, 49), 0.0)]
[((51, 59), 0.0), ((52, 58), 0.0), ((53, 57), 0.0), ((54, 56), 0.0), ((55, 55), 0.0), ((56, 54), 0.0), ((57, 53), 0.0), ((58, 52), 0.0), ((59, 51), 0.0), ((60, 50), 0.0), ((39, 50), 0.0), ((40, 49), 0.0), ((40, 51), 0.0), ((41, 48), 0.0), ((41, 52), 0.0), ((42, 47), 0.0), ((42, 53), 0

[((54, 56), 0.0), ((55, 55), 0.0), ((56, 54), 0.0), ((57, 53), 0.0), ((58, 52), 0.0), ((59, 51), 0.0), ((60, 50), 0.0), ((39, 50), 0.0), ((40, 49), 0.0), ((40, 51), 0.0), ((41, 48), 0.0), ((41, 52), 0.0), ((42, 47), 0.0), ((42, 53), 0.0), ((43, 46), 0.0), ((43, 54), 0.0), ((44, 45), 0.0), ((44, 55), 0.0), ((45, 44), 0.0), ((45, 56), 0.0), ((46, 43), 0.0), ((46, 57), 0.0), ((47, 42), 0.0), ((47, 58), 0.0), ((48, 41), 0.0), ((48, 59), 0.0), ((49, 40), 0.0), ((49, 60), 0.0), ((50, 39), 0.0), ((51, 40), 0.0), ((52, 41), 0.0), ((53, 42), 0.0), ((54, 43), 0.0), ((55, 44), 0.0), ((56, 45), 0.0), ((57, 46), 0.0), ((58, 47), 0.0), ((59, 48), 0.0), ((60, 49), 0.0), ((50, 61), 0.0), ((51, 60), 0.0), ((52, 59), 0.0), ((53, 58), 0.0)]
[((54, 56), 0.0), ((55, 55), 0.0), ((56, 54), 0.0), ((57, 53), 0.0), ((58, 52), 0.0), ((59, 51), 0.0), ((60, 50), 0.0), ((39, 50), 0.0), ((40, 49), 0.0), ((40, 51), 0.0), ((41, 48), 0.0), ((41, 52), 0.0), ((42, 47), 0.0), ((42, 53), 0.0), ((43, 46), 0.0), ((43, 54), 0

In [52]:
p_result[()]

{(39, 50): <__main__.vertex_i instance at 0x108b0f368>,
 (40, 49): <__main__.vertex_i instance at 0x108b0fc20>,
 (40, 50): <__main__.vertex_i instance at 0x108b263f8>,
 (40, 51): <__main__.vertex_i instance at 0x108b0fdd0>,
 (41, 48): <__main__.vertex_i instance at 0x108b0f320>,
 (41, 49): <__main__.vertex_i instance at 0x108b26ab8>,
 (41, 50): <__main__.vertex_i instance at 0x108afe3f8>,
 (41, 51): <__main__.vertex_i instance at 0x108b14320>,
 (41, 52): <__main__.vertex_i instance at 0x108b47758>,
 (42, 47): <__main__.vertex_i instance at 0x108b47a28>,
 (42, 48): <__main__.vertex_i instance at 0x108b14950>,
 (42, 49): <__main__.vertex_i instance at 0x108c25248>,
 (42, 50): <__main__.vertex_i instance at 0x1038e23b0>,
 (42, 51): <__main__.vertex_i instance at 0x108c254d0>,
 (42, 52): <__main__.vertex_i instance at 0x108b147e8>,
 (42, 53): <__main__.vertex_i instance at 0x108b47cf8>,
 (43, 46): <__main__.vertex_i instance at 0x108b47d40>,
 (43, 47): <__main__.vertex_i instance at 0x108b